In [0]:
import torch
import torch_geometric
import numpy as np

In [0]:
from torch.nn import Sequential as Seq, Linear as Lin, ReLU
from torch_geometric.nn import MessagePassing

class EdgeConv(MessagePassing):
    def __init__(self, F_in, F_out):
        super(EdgeConv, self).__init__(aggr='max',flow = 'target_to_source')  # "Max" aggregation.
        self.mlp = Seq(Lin(2 * F_in, F_out), ReLU(), Lin(F_out, F_out))

    def forward(self, x,edge_index):
        # x has shape [N, F_in]
        # edge_index has shape [2, E]
        print('in forward x is :{}'.format(x))
        h = x*1.5
        return self.propagate(edge_index, x=x, h = h)  # shape [N, F_out]

    def message(self, x_i, x_j,h, h_i, h_j):
        print('x_i:{}'.format(x_i))
        print('x_j:{}'.format(x_j))
        print('h  :{}'.format(h))
        print('h i :{}'.format(h_i))
        print('h j :{}'.format(h_j))
        # x_i has shape [E, F_in]
        # x_j has shape [E, F_in]
        edge_features = torch.cat([x_i, x_j - x_i], dim=1)  # shape [E, 2 * F_in]
        print('edge_features :{}'.format(edge_features))
        print('mlp edge_fearutes :{}'.format(self.mlp(edge_features)))
        return self.mlp(edge_features)  # shape [E, F_out]

In [74]:
# in_channels = 16
# edge_index = torch.tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3],
#                                [1, 2, 3, 0, 2, 3, 0, 1, 3, 0, 1, 2]])
# num_nodes = edge_index.max().item() + 1
# x = torch.randn((num_nodes,in_channels))

# random seed 
SEED = 110 
np.random.seed(SEED) 
torch.manual_seed(SEED) 
torch.cuda.manual_seed(SEED)

edge_index_ = torch.tensor([[0,1,1,2,2,3],[1,0,2,1,3,2]])
#'target_to_source'
x = torch.randn((4,2))

print('node feature:{} shape {}'.format(x, x.size()))
print('node num is :{}'.format(x.size()[0]))
edge_conv = EdgeConv(x.size()[1], x.size()[1]*2)
hidden_feature = edge_conv(x, edge_index_)
print('node feature:{} shape {}'.format(hidden_feature, hidden_feature.size()))



node feature:tensor([[-0.4542, -1.1170],
        [ 0.7534,  2.6707],
        [-1.2473, -0.8668],
        [ 0.1647, -0.1978]]) shape torch.Size([4, 2])
node num is :4
in forward x is :tensor([[-0.4542, -1.1170],
        [ 0.7534,  2.6707],
        [-1.2473, -0.8668],
        [ 0.1647, -0.1978]])
x_i:tensor([[-0.4542, -1.1170],
        [ 0.7534,  2.6707],
        [ 0.7534,  2.6707],
        [-1.2473, -0.8668],
        [-1.2473, -0.8668],
        [ 0.1647, -0.1978]])
x_j:tensor([[ 0.7534,  2.6707],
        [-0.4542, -1.1170],
        [-1.2473, -0.8668],
        [ 0.7534,  2.6707],
        [ 0.1647, -0.1978],
        [-1.2473, -0.8668]])
h  :tensor([[-0.6812, -1.6755],
        [ 1.1302,  4.0060],
        [-1.8710, -1.3002],
        [ 0.2470, -0.2968]])
h i :tensor([[-0.6812, -1.6755],
        [ 1.1302,  4.0060],
        [ 1.1302,  4.0060],
        [-1.8710, -1.3002],
        [-1.8710, -1.3002],
        [ 0.2470, -0.2968]])
h j :tensor([[ 1.1302,  4.0060],
        [-0.6812, -1.6755],
      

In [53]:
#edge_index_ = torch.tensor([[0,1,1,2],[1,2,3,3]])
for parameters in edge_conv.named_parameters():
    print(parameters)

('mlp.0.weight', Parameter containing:
tensor([[ 0.2339, -0.1264, -0.0869,  0.0030, -0.0564, -0.2219, -0.1512, -0.2413,
          0.2284, -0.2534],
        [ 0.2174,  0.0612,  0.1780, -0.1818, -0.0561, -0.2301,  0.2644,  0.2187,
          0.0921, -0.2705],
        [ 0.1420, -0.3138,  0.2522, -0.0327,  0.2440, -0.2843, -0.1267,  0.1805,
          0.1376,  0.1444],
        [ 0.2422,  0.2226,  0.2828,  0.0410,  0.0326,  0.1039, -0.1294,  0.1071,
          0.0511,  0.1972],
        [-0.2804,  0.0284,  0.1446,  0.1746, -0.2075,  0.0478,  0.1570,  0.2182,
          0.2310,  0.2890],
        [-0.2611, -0.1077, -0.0249,  0.1980, -0.1131,  0.1167,  0.1708, -0.2562,
          0.1654, -0.2475],
        [-0.2213,  0.1678, -0.0679,  0.3111,  0.3126,  0.2350,  0.2912,  0.1298,
          0.2645, -0.1921],
        [ 0.0991, -0.0082,  0.2347, -0.2957, -0.1309,  0.1926,  0.0125, -0.0239,
          0.1034,  0.0145],
        [ 0.1885, -0.1790,  0.3056,  0.0712,  0.2580, -0.1210, -0.0615, -0.0611,
        

In [0]:

from torch_geometric.nn import SAGEConv
SEED = 110 
np.random.seed(SEED) 
torch.manual_seed(SEED) 
torch.cuda.manual_seed(SEED)
data = torch.randn((4,5))
in_channels = data.size(1)


conv1 = SAGEConv(in_channels, in_channels*2, normalize=True, bias=True)


edge_index = torch.tensor([[0,1,2,3],[1,2,0,1]])
out1 = conv1(data,edge_index)
print(out1)

tensor([[-0.1716,  0.0126, -0.1496,  0.3717, -0.3207,  0.2212, -0.0057, -0.6195,
          0.3733, -0.3672],
        [ 0.1158,  0.0813, -0.0526,  0.1341, -0.2646,  0.0453,  0.1727, -0.5094,
          0.3050, -0.7105],
        [-0.4656, -0.2441, -0.3872,  0.1599, -0.2008,  0.0641,  0.2571, -0.4470,
          0.0925, -0.4789],
        [-0.0220,  0.5372,  0.1183,  0.2815, -0.3936,  0.4917,  0.1792, -0.3118,
          0.1870, -0.2382]], grad_fn=<DivBackward0>)


In [0]:
for parameters in conv1.named_parameters():
    print(parameters)


('weight', Parameter containing:
tensor([[ 0.3308, -0.1787, -0.1229,  0.0042, -0.0797, -0.3138, -0.2138, -0.3413,
          0.3229, -0.3584],
        [ 0.3074,  0.0865,  0.2517, -0.2572, -0.0793, -0.3254,  0.3739,  0.3093,
          0.1302, -0.3825],
        [ 0.2008, -0.4437,  0.3567, -0.0463,  0.3451, -0.4021, -0.1792,  0.2553,
          0.1946,  0.2043],
        [ 0.3425,  0.3147,  0.4000,  0.0580,  0.0461,  0.1469, -0.1831,  0.1515,
          0.0723,  0.2788],
        [-0.3966,  0.0401,  0.2044,  0.2469, -0.2935,  0.0677,  0.2220,  0.3087,
          0.3266,  0.4087]], requires_grad=True))
('bias', Parameter containing:
tensor([-0.3693, -0.1523, -0.0352,  0.2800, -0.1599,  0.1650,  0.2416, -0.3623,
         0.2339, -0.3501], requires_grad=True))
